# [AI Essential] 심층신경망 워크샵

## 0. 환경 설정
- 필요 모듈 설치

In [ ]:
!pip install torchinfo boston JAEN -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 354.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.5/174.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.0/397.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.5/51.5 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 

- 필요 모듈 import

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary
from boston import load_boston
from torch.utils.data import DataLoader, TensorDataset

# 장치 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## Boston Dataset

##### 개요
- Boston 데이터 세트는 주택 가격 예측을 위한 데이터 세트 (회귀 분석 문제)
- 보스턴의 여러 구역에서 주택 가격에 영향을 미치는 다양한 요인들을 포함하고 있음

##### 주요 특징
- 목적: 주어진 특성을 바탕으로 보스턴 각 지역의 주택 가격(중앙값)을 예측하는 것.
- 총 샘플 수: 506개의 데이터 포인트.
- 특성 수: 13개의 설명 변수(특성)와 1개의 목표 변수(주택 가격).

##### 특성
- CRIM: 자치시(town) 별 1인당 범죄율.
- ZN: 25,000 평방피트 이상의 주거지역 비율.
- INDUS: 비소매 상업 지역 비율(도시 별).
- CHAS: 찰스 강과 인접 여부(1: 강 인접, 0: 인접하지 않음).
- NOX: 일산화질소 농도 (백만 분의 일).
- RM: 주택당 평균 방 개수.
- AGE: 1940년 이전에 건축된 소유 주택 비율.
- DIS: 5개의 보스턴 고용센터까지의 가중 거리.
- RAD: 방사형 고속도로 접근성 지수.
- TAX: 10,000달러당 재산세율.
- PTRATIO: 지역별 학생-교사 비율.
- B: 지역별 흑인 비율.
- LSTAT: 하위 계층의 비율.
- MEDV: 주택 가격의 중앙값(단위: 1,000달러).

## 1. 데이터 불러오기 및 전처리

In [ ]:
# 데이터셋 로드
data = load_boston()
X, y = torch.tensor(data.data, dtype=torch.float32), torch.tensor(data.target, dtype=torch.float32).view(-1, 1)
X.shape, y.shape

(torch.Size([506, 13]), torch.Size([506, 1]))

In [ ]:
# 특성 데이터 표준화
X_means = X.mean(dim=0)
X_stds = X.std(dim=0)

X = (X-X_means)/X_stds
X.mean(dim=0), X.std(dim=0)

(tensor([-1.6020e-08, -1.5078e-08, -1.3382e-07, -1.1308e-08,  4.2854e-07,
          3.9202e-07,  1.2486e-07, -3.3925e-08,  7.2562e-08,  9.4237e-09,
         -5.3244e-08, -4.6576e-07,  5.5600e-08]),
 tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000]))

In [ ]:
# 학습 및 테스트 데이터셋 분리
torch.manual_seed(42)
indeces = torch.randperm(506)
test_rate = 0.2
test_size = int(X.shape[0]*test_rate)
X_train, X_test = X[indeces][:-test_size], X[indeces][-test_size:]
y_train, y_test = y[indeces][:-test_size], y[indeces][-test_size:]
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([405, 13]),
 torch.Size([101, 13]),
 torch.Size([405, 1]),
 torch.Size([101, 1]))

In [ ]:
# Dataset 및 DataLoader 생성
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=32, shuffle=False)

In [ ]:
# DataLoader 사용 예시
for batch_idx, (data, target) in enumerate(train_loader):
    print(f"Batch {batch_idx+1}")
    print("Data:", data)
    print("Target:", target)
    break  # 한 번만 출력

Batch 1
Data: tensor([[ 4.8983, -0.4872,  1.0150, -0.2723,  1.1935, -2.5129,  1.1164, -1.0148,
          1.6596,  1.5294,  0.8058, -2.9400,  3.4066],
        [ 0.4913, -0.4872,  1.0150, -0.2723,  0.8656, -0.1076, -0.1128, -0.3949,
          1.6596,  1.5294,  0.8058,  0.4406,  0.0794],
        [-0.1702, -0.4872,  1.2307, -0.2723,  2.7296, -0.8193,  1.0631, -1.0314,
         -0.5225, -0.0311, -1.7347, -1.0376,  0.4393],
        [-0.4158,  0.3703, -1.1380, -0.2723, -0.9648,  0.9726, -1.1146,  0.6884,
         -0.5225, -1.1406, -1.6423,  0.3895, -1.1291],
        [-0.3459, -0.4872, -0.4368, -0.2723, -0.1441, -0.2685,  0.5657,  0.3167,
         -0.6373, -0.6007,  1.1753,  0.2557, -0.3351],
        [-0.4135,  1.2278, -0.6890,  3.6648, -0.9294,  0.6737, -1.2674,  0.1342,
         -0.6373, -0.9152, -0.3952,  0.4406, -1.2775],
        [-0.4109, -0.4872,  0.2468, -0.2723, -1.0157, -0.0166, -2.2230,  0.2168,
         -0.5225, -0.0607,  0.1129,  0.4189, -0.8224],
        [-0.3282, -0.4872, -0.1803

## 2. 모델 정의

In [ ]:
# 심층 신경망(DNN) 모델 정의
class DNNModel(nn.Module):
    def __init__(self):
        super(DNNModel, self).__init__()
        # 첫 번째 은닉층
        self.fc1 = nn.Linear(13, 32)
        self.relu1 = nn.ReLU()  # 활성화 함수

        # 두 번째 은닉층
        self.fc2 = nn.Linear(32, 64)
        self.relu2 = nn.ReLU()  # 활성화 함수

        # 세 번째 은닉층
        self.fc3 = nn.Linear(64, 128)
        self.relu3 = nn.ReLU()  # 활성화 함수

        # 출력층 (회귀 문제이므로 활성화 함수 없음)
        self.fc4 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)

        x = self.fc2(x)
        x = self.relu2(x)

        x = self.fc3(x)
        x = self.relu3(x)

        x = self.fc4(x)
        return x

model = DNNModel().to(device)
summary(model, (32, 13))

Layer (type:depth-idx)                   Output Shape              Param #
DNNModel                                 [32, 1]                   --
├─Linear: 1-1                            [32, 32]                  448
├─ReLU: 1-2                              [32, 32]                  --
├─Linear: 1-3                            [32, 64]                  2,112
├─ReLU: 1-4                              [32, 64]                  --
├─Linear: 1-5                            [32, 128]                 8,320
├─ReLU: 1-6                              [32, 128]                 --
├─Linear: 1-7                            [32, 1]                   129
Total params: 11,009
Trainable params: 11,009
Non-trainable params: 0
Total mult-adds (M): 0.35
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 0.04
Estimated Total Size (MB): 0.10

## 3. 손실함수 및 옵티마이저 정의

In [ ]:
# 손실 함수와 옵티마이저 정의
lr = 0.001
criterion = nn.MSELoss()  # 손실 함수: 평균 제곱 오차 (MSE)
optimizer = optim.Adam(model.parameters(), lr=lr)  # 옵티마이저: Adam

## 4. 학습 및 평가함수 정의

In [ ]:
num_epochs = 100

for epoch in range(num_epochs):
    model.train()  # 모델을 학습 모드로 전환
    train_loss = 0.0

    # 4.1. 학습 과정
    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # 데이터를 device로 이동
        optimizer.zero_grad()  # 이전에 계산한 gradients를 초기화

        # 순전파
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # 역전파
        loss.backward()
        optimizer.step()

        train_loss += loss * inputs.shape[0] / len(train_loader.dataset)

    # 4.2. 평가 과정 (매 에폭마다 테스트 데이터에서 성능을 측정)
    model.eval()  # 모델을 평가 모드로 전환
    test_loss = 0.0
    predictions = []
    actuals = []

    with torch.no_grad():  # 평가 시에는 gradients 계산하지 않음
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)  # 데이터를 device로 이동
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss * inputs.shape[0] / len(test_loader.dataset)

            # 예측값과 실제값을 저장 (평가용)
            predictions.append(outputs)
            actuals.append(targets)

    # 4.3. 에폭 결과 출력
    if epoch % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]")
        print(f"Train Loss: {train_loss:.4f}")
        print(f"Test Loss: {test_loss:.4f}")
        print("-" * 50)

Epoch [1/100]
Train Loss: 578.0305
Test Loss: 597.4517
--------------------------------------------------
Epoch [11/100]
Train Loss: 19.4018
Test Loss: 28.9212
--------------------------------------------------
Epoch [21/100]
Train Loss: 13.6568
Test Loss: 21.6225
--------------------------------------------------
Epoch [31/100]
Train Loss: 11.4687
Test Loss: 18.9591
--------------------------------------------------
Epoch [41/100]
Train Loss: 10.1299
Test Loss: 17.4133
--------------------------------------------------
Epoch [51/100]
Train Loss: 9.4021
Test Loss: 16.1317
--------------------------------------------------
Epoch [61/100]
Train Loss: 8.7274
Test Loss: 15.2265
--------------------------------------------------
Epoch [71/100]
Train Loss: 8.2970
Test Loss: 13.6728
--------------------------------------------------
Epoch [81/100]
Train Loss: 7.9382
Test Loss: 12.9467
--------------------------------------------------
Epoch [91/100]
Train Loss: 7.3253
Test Loss: 12.9552
-----

In [ ]:
model.eval()  # 모델을 평가 모드로 전환
test_loss = 0.0
predictions = []
actuals = []

with torch.no_grad():  # 평가 시에는 gradients 계산하지 않음
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)  # 데이터를 device로 이동
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        test_loss += loss * inputs.shape[0] / len(test_loader.dataset)

        # 예측 값 생성
        predictions.append(outputs)
        actuals.append(targets)

In [ ]:
# mse, rmse, mae 계산
predictions = torch.cat(predictions)
actuals = torch.cat(actuals)
print(f'mse: {((actuals-predictions)**2).mean().item():.4f}')
print(f'rmse: {((actuals-predictions)**2).mean().sqrt().item():.4f}')
print(f'mae: {(actuals-predictions).abs().mean().item():.4f}')

mse: 12.5206
rmse: 3.5385
mae: 2.1680


In [ ]:
# 사이킷런
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error

predictions = predictions.cpu().numpy()
actuals = actuals.cpu().numpy()
print(f'mse: {mean_squared_error(actuals, predictions):.4f}')
print(f'rmse: {root_mean_squared_error(actuals, predictions):.4f}')
print(f'mae: {mean_absolute_error(actuals, predictions):.4f}')

mse: 12.5206
rmse: 3.5385
mae: 2.1680
